In [1]:
with open("../data/reliable_news_prep", "r") as reliable_file:
    rel = [line.strip() for line in reliable_file]
with open("../data/fake_news_prep", "r") as fake_file:
    fake = [line.strip() for line in fake_file]

In [2]:
length=50000

In [3]:
text = rel[:length]+fake[:length]

In [4]:
labels = [0 if i<length else 1 for i in range(2*length)] # reliable - 0; fake - 1

In [5]:
from training_preprocess import prepare_fastText_embedding_matrix as prepare_embedding
from training_preprocess import sequence_vectorize
from training_preprocess import train_val_test_split as split

/home/miniconda3/envs/deep_learning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/miniconda3/envs/deep_learning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/miniconda3/envs/deep_learning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/miniconda3/envs/deep_learning/lib/pyt

In [18]:
X_train, X_val, X_test, Y_train, Y_test, Y_val = split(text, labels, 0.2, 0.1)

In [7]:
x_train, x_val, x_test, word_index = sequence_vectorize(X_train, X_val, X_test, sequence_length=1100)

In [8]:
embedding_matrix = prepare_embedding(word_index, "../data/cc.en.300.bin.gz")

In [9]:
from keras import layers, models, optimizers, regularizers

Using TensorFlow backend.


In [10]:
import numpy as np

def label_to_vector(labels):
    array = np.zeros([len(labels), 2])
    for i, label in enumerate(labels):
        array[i, label] = 1
    return array

In [11]:
def bidirectional_LSTM(embedding_dim, dropout_rate, input_shape, num_classes, is_embedding_trainable, embedding_matrix):
    
    # Add an Input Layer
    input_layer = layers.Input((input_shape, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(input_dim=embedding_matrix.shape[0],
                                       output_dim=embedding_dim,
                                       weights=[embedding_matrix],
                                       trainable=is_embedding_trainable)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.6)(embedding_layer)
    
    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.LSTM(20))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(70, activation="relu")(lstm_layer)
    output_layer2 = layers.Dropout(dropout_rate)(output_layer1)
    output_layer3 = layers.Dense(num_classes, activation="softmax")(output_layer2)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer3)
    
    return model

In [19]:
y_train = label_to_vector(Y_train)

In [20]:
y_val = label_to_vector(Y_val)

In [14]:
import keras

model = bidirectional_LSTM(300, 0.2, 1100, 2, True, embedding_matrix)
model.compile(optimizer='Adam', loss="binary_crossentropy", metrics=['accuracy'])

callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]

model.fit(x_train,
          y_train,
          epochs=10,
          callbacks=callbacks,
          validation_data=(x_val, y_val),
          verbose=2,  # Logs once per epoch.
          batch_size=128)

model.save_weights('../data/sequence_model_with_pre_trained_embedding.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 70000 samples, validate on 20000 samples
Epoch 1/10
 - 3173s - loss: 0.1929 - acc: 0.9244 - val_loss: 0.1021 - val_acc: 0.9665
Epoch 2/10
 - 3176s - loss: 0.0812 - acc: 0.9722 - val_loss: 0.0774 - val_acc: 0.9729
Epoch 3/10
 - 3180s - loss: 0.0634 - acc: 0.9788 - val_loss: 0.0768 - val_acc: 0.9751
Epoch 4/10
 - 3175s - loss: 0.0452 - acc: 0.9859 - val_loss: 0.0840 - val_acc: 0.9746
Epoch 5/10
 - 3178s - loss: 0.0338 - acc: 0.9891 - val_loss: 0.0906 - val_acc: 0.9758
Epoch 6/10
 - 3178s - loss: 0.0702 - acc: 0.9735 - val_loss: 0.0877 - val_acc: 0.9755


In [21]:
y_test = label_to_vector(Y_test)

In [16]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 452s 45ms/step


[0.0924303956180811, 0.9742]